# Exercise03 Shape Grammar to Pix
## Exercise Basic
* Label 4 facades from Zurich to expand! the training data for the pix2pix approach. Use the same colors as in the examples
* Generate 8 individual facade pattern (with a shape grammar or a strategy of your choice) and apply a pix2pix facade on them.

Submission: 
* 4 labeled images with the ground truth. **Those images don't need to be included in the dataset**! YourLastName_Train_XX.jpg. Example:

![alt text](https://docs.google.com/drawings/d/e/2PACX-1vQcxtbPqMQIUfW2Ajyy7bJLTZL8QTcEcNvM2IQ7lrdqfyoKyHSUHMJKMrhaJHF176AR8938jHcQwVpB/pub?w=511&h=258)
* 8 generated colored facade pattern YourLastName_Test_XX.jpg. Example:

![alt text](https://docs.google.com/drawings/d/e/2PACX-1vRHF_PLeD2cwFosh12NVVdEVsWBzgm48WDGZboUsNtSuOMJ9EB2YUnbQB3o-KEWfP-OQjYA5CU6kFFI/pub?w=516&h=258)
* 8 resulting pix2pix textured facade images. YourLastName_Generate_XX.jpg. Example:

![alt text](https://docs.google.com/drawings/d/e/2PACX-1vSXWOvVcvd4VODzX0D8CUY_Bj_VCvjWrG-3qTqvXm0dc6Ccj0IR3sMxkj1Fln_h0dwLl9XBJvn-7FrI/pub?w=962&h=301)

## Exercise Plus
Generate 8 individual pattern for another pix2pix dataset, for example the topview-city dataset.
* 8 generated  pattern and 8 resulting pix2pix transformations. YourLastName_Generate_YourDataset_XX.jpg

In [0]:
!rm -rf mola
!git clone https://github.com/dbt-ethz/mola.git

Cloning into 'mola'...
remote: Enumerating objects: 124, done.
remote: Counting objects: 100% (124/124), done.
remote: Compressing objects: 100% (119/119), done.
remote: Total 1405 (delta 65), reused 13 (delta 4), pack-reused 1281
Receiving objects: 100% (1405/1405), 24.69 MiB | 14.84 MiB/s, done.
Resolving deltas: 100% (891/891), done.


## Import

In [0]:
from IPython.display import HTML, SVG
import mola.renderP5JS as r2d
from mola.core import *
from mola import subdivision
import random
from google.colab import files
import mola.renderBabylonJS as renderer3D
from mola.core import *
from mola import factory as mf
from mola import color
from mola import io

## Labels and Colors taken from CMP Facade Database
http://cmp.felk.cvut.cz/~tylecr1/facade/

In [0]:
labelColors={
  "background": (0,0,222),
  "facade": (0,48,255),
  "molding": (255,80,0),
  "cornice":(32,255,215),
  "pillar":(255,0,0),
  "window": (0,128,255),
  "door":(0,207,255),
  "sill":(115,254,139),
  "blind":(255,238,0),
  "balcony":(192,255,62),
  "shop":(174,0,0),
  "deco":(255,160,0)
}

In [0]:
r2d.beginDraw(width=256,height=256)
r2d.noStroke()
w = 256/len(labelColors.values())
for i, (k, v) in enumerate(labelColors.items()):
  r2d.fill(v)
  r2d.rect(i * w,0,(i+1)*w,256)
HTML(r2d.endDraw())

In [0]:
# Color Faces by Label
def colorAllFaces(faces):
  for f in faces:
     ccolor = labelColors[f.group]
     if ccolor!=None:
      f.color = ccolor

##Draw basic building shape and label faces

In [1]:
W = H = 256

faces=[]

x1 = 4.9 #@param {type:"slider", min:3, max:7, step:0.1}
x2 = 7 #@param {type:"slider", min:3, max:20, step:0.1}

#initial vertices of building
perimeter_vertices = [Vertex(W/x1,H),Vertex(W/x2,H/3),Vertex(W/2,H/20),Vertex((x2-1)*W/x2,H/3),Vertex((x1-1)*W/x1,H)]
center_vertex = Vertex(W/2,H/2.5)

for i in range(len(perimeter_vertices)):
  f = Face([center_vertex, perimeter_vertices[i], perimeter_vertices[(i+1)%len(perimeter_vertices)]])
  if(i==4):
    f.group = "door"
  elif(i==1 or i==2):
    f.group = "molding"
  else:
    f.group = "facade"
  faces.append(f)

r2d.beginDraw(width=W,height=H)
r2d.noStroke()
bColor=labelColors["background"]
r2d.background(bColor[0],bColor[1],bColor[2])
colorAllFaces(faces)
r2d.displayFaces2D(faces)
HTML(r2d.endDraw())

NameError: ignored

##Subdivide with splitOffset, splitGrid, splitRel to generate more building parts 

In [0]:
newFaces = []
for i, f in enumerate(faces):
  if(f.group=="door"):
    offsetFaces = subdivision.splitOffset(f,10)
    for j,f in enumerate(offsetFaces):
      if(j==(len(offsetFaces)-1)):
        f.group = "door"
      else:
        f.group = "pillar"
    newFaces.extend(offsetFaces)
    
  elif(f.group=="facade"):
    gridFaces = subdivision.splitGrid(f,3,3)
    for gf in gridFaces:
      if(len(gf.vertices)==4):
        gf.vertices.reverse()
      goffsetFaces = subdivision.splitOffset(gf,4)
      for j,f in enumerate(goffsetFaces):
        if(j==(len(goffsetFaces)-1)):
          f.group = "window"
        elif(j==1):
          f.group = "cornice"
        else:
          f.group = "facade"
      newFaces.extend(goffsetFaces)  
  
  elif(f.group=="molding"):
    newFaces2 = []
    offsetFaces = subdivision.splitOffset(f,10)
    for j,f in enumerate(offsetFaces):
      if(j==3):
        gridFaces = subdivision.splitGrid(f,1,3)
        for k,gf in enumerate(gridFaces):
          if(k==0):
            gf.group="balcony"
          elif(k==1):
            gf.group="blind"
          else:
            gf.group="window"
        newFaces2.extend(gridFaces) 
      else:
        f.group = "molding"
        newFaces2.append(f)
    newFaces.extend(newFaces2)
  else:
    newFaces.append(f)

faces = newFaces

r2d.beginDraw(width=W,height=H)
r2d.noStroke()#stroke(255,0,0)
bColor=labelColors["background"]
r2d.background(bColor[0],bColor[1],bColor[2])
colorAllFaces(faces)
r2d.displayFaces2D(faces)
HTML(r2d.endDraw())

In [0]:
newFaces = []
for f in faces:
  if(f.group=="window" and len(f.vertices)==4):
    splitFaces = subdivision.splitRel(f,0,random.random()/2 + 0.5)
    splitFaces[0].group="window"
    splitFaces[1].group="balcony"
    newFaces.extend(splitFaces)
  else:
    newFaces.append(f)
faces = newFaces

newFaces = []
for f in faces:
  if(f.group=="window" and len(f.vertices)==4):
    splitFaces = subdivision.splitRel(f,-1,random.random())
    splitFaces[0].group="window"
    splitFaces[1].group="blind"
    newFaces.extend(splitFaces)
  else:
    newFaces.append(f)
faces = newFaces

newFaces = []
for f in faces:
  if(f.group=="door"):
    gridFaces = subdivision.splitGrid(f,1,3)
    for i, gf in enumerate(gridFaces):
      if(i==1):
        gf.group="door"
      else:
        gf.group="deco"
    newFaces.extend(gridFaces)
  else:
    newFaces.append(f)
faces = newFaces

In [0]:
r2d.beginDraw(width=W,height=H)
r2d.noStroke()#stroke(255,0,0)
bColor=labelColors["background"]
r2d.background(bColor[0],bColor[1],bColor[2])
colorAllFaces(faces)
r2d.displayFaces2D(faces)
HTML(r2d.endDraw())

In [0]:
faces=[]
myFaces=Face([Vertex(0,0),Vertex(100,0),Vertex(100,200),Vertex(0,200)])
myFaces.group="window"
faces.append(myFaces)
colorAllFaces(faces)
r2d.beginDraw(256,256)
r2d.background(0,0,222)
r2d.displayFaces2D(faces)
HTML(r2d.endDraw())


In [0]:
#@title GUI { form-width: "20%" }
save_images = False #@param {type:"boolean"}

faces=[]
dX=120
dY=200
cX=128
myFaces=Face([Vertex(cX-dX/2.0,0),Vertex(cX+dX/2.0,0),Vertex(cX+dX/2.0,dY),Vertex(cX-dX/2.0,dY)])
myFaces.group="window"
faces.append(myFaces)

r2d.beginDraw(256,256)
r2d.translate(0,256) # drawing has y axis downwards, here we adjust this
r2d.scale(1,-1)

for j in range(3):
  newFaces=[]
  for f in faces:
    splitFaces=subdivision.splitGrid(f,4,2)
    for i,newFace in enumerate(splitFaces):
      if i%3==0:
        newFace.group="window"
      else:
        newFace.group="pillar"
    newFaces.extend(splitFaces)
  faces=newFaces
  colorAllFaces(faces)
  r2d.background(0,0,222)
  r2d.displayFaces2D(faces)
  if save_images:
    r2d.saveCanvas('myCanvas'+str(j)+'.jpg')
HTML(r2d.endDraw())
